# Final Tuning of LLM


### Installing the required libraries

In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install peft
!pip install trl
!pip install bitsandbytes
!pip install evaluate

### Importing the required libraries

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, DefaultDataCollator , AutoModelForQuestionAnswering, AutoModelForCausalLM, TrainingArguments, Trainer, pipeline, BitsAndBytesConfig
from trl import SFTTrainer
from evaluate import load
from peft import LoraConfig, get_peft_model

### Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = '/content/drive/MyDrive/ATAI/'

### Pre Processing functions

In [ ]:
def performance_metrics(model, tokenizer):
  question_answerer = pipeline("question-answering", model=model, tokenizer = tokenizer)

  predictions = []
  references = []

  for example in squad_test:
      # Get the prediction for the current example
      prediction = question_answerer(context = example['context'], question = example['question'])

      # Store the prediction in the predictions list
      predictions.append({
          'prediction_text': prediction['answer'],
          'id': example['id']
      })

      # Store the reference in the references list
      references.append({
          'answers': {
              'answer_start': example['answers']['answer_start'],
              'text': example['answers']['text']
          },
          'id': example['id']
      })

  print(squad_metric.compute(predictions=predictions, references=references))

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

### Data collection

In [ ]:
squad = load_dataset("squad", split="train")

In [ ]:
squad = squad.train_test_split(test_size=0.2)

### Preprocessing

#### Loading the Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenizer_bert = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
tokenized_squad = squad.map(preprocess_function, batched=True)

#### Data Collator

In [ ]:
data_collator = DefaultDataCollator()

### Model Selection

#### Loading the DistillBERT model

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

In [ ]:
performance_metrics(model, tokenizer)

{'exact_match': 0.0, 'f1': 4.1435612426788895}


## Fine-Tuning of the models

### Fine tuning for 2 epoch

In [ ]:
training_args_1 = TrainingArguments(
    output_dir=path,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    logging_steps=500,
    weight_decay=0.01
)

trainer_1 = Trainer(
    model=model,
    args=training_args_1,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer_1.train()

#trainer_1.model.save_pretrained(path + '/FT_1/')
model_ft_1 = AutoModelForQuestionAnswering.from_pretrained(path + '/FT_1/')

#### Evaluating

In [ ]:
performance_metrics(model_ft_1, tokenizer)

### Fine tuning for 2 epoch - Warm up

In [ ]:
training_args = TrainingArguments(
    output_dir=path,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    logging_steps=500,
    weight_decay=0.01,
    warmup_ratio=0.03,
)

trainer_wu = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer_wu.train()

#trainer_wu.model.save_pretrained(path + '/WU/')
model_ft_wu = AutoModelForQuestionAnswering.from_pretrained(path + '/WU/')

#### Evaluating

In [ ]:
model_ft_wu = AutoModelForQuestionAnswering.from_pretrained(path + '/WU/')

In [ ]:
performance_metrics(model_ft_wu, tokenizer)

### Fine tuning for 4 epoch

In [ ]:
training_args = TrainingArguments(
    output_dir=path,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    logging_steps=500,
    weight_decay=0.01
)

trainer_4 = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer_4.train()

trainer_4.model.save_pretrained(path + '/FT_4/')
model_ft_4 = AutoModelForQuestionAnswering.from_pretrained(path + '/FT_4/')

#### Evaluating

In [ ]:
performance_metrics(model_ft_4, tokenizer)

### Reduced Dataset

In [ ]:
squad = load_dataset("squad", split="train[:5000]")

In [ ]:
squad = squad.train_test_split(test_size=0.2)

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", trust_remote_code=True)
tokenized_squad = squad.map(preprocess_function, batched=True)

In [ ]:
training_args = TrainingArguments(
    output_dir=path,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    logging_steps=50,
    weight_decay=0.01
)

trainer_rd = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer_rd.train()

#trainer_rd.model.save_pretrained(path + '/RD/')
model_ft_rd = AutoModelForQuestionAnswering.from_pretrained(path + '/RD/')

#### Evaluating

In [ ]:
model_ft_rd = AutoModelForQuestionAnswering.from_pretrained(path + '/RD/')

In [ ]:
performance_metrics(model_ft_rd, tokenizer)

### BERT

In [ ]:
tokenizer_bert = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenized_squad = squad.map(preprocess_function, batched=True)

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir=path,
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    logging_steps=500,
    weight_decay=0.01
)

trainer_bert = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer_bert,
    data_collator=data_collator,
)

trainer_bert.train()
trainer_bert.model.save_pretrained(path + '/FT_bert/')
model_bert = AutoModelForQuestionAnswering.from_pretrained(path + '/FT_bert/')

#### Evaluating

In [ ]:
performance_metrics(model_bert, tokenizer_bert)

{'exact_match': 59.3, 'f1': 64.89730164762767}
